In [ ]:
!pip install -U transformers datasets peft accelerate bitsandbytes trl huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

# Hugging Face 로그인
notebook_login()

In [3]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [4]:
model_id = 'LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct'
output_dir = "./exaone-customer-anlysis"

In [ ]:
# 4-bit 양자화된 모델 로드를 위한 설정
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',             # 양자화 타입 지정
    bnb_4bit_use_double_quant=True,        # 이중 양자화(양자화된 가중치를 한번 더 양자화)
    bnb_4bit_compute_dtype=torch.bfloat16  # 16bit의 부동소수점 데이터 타입 사용
)

# 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
)

In [6]:
# 모델 학습 준비
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=16, 
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

In [7]:
SYSTEM_PROMPT = """
카드사 콜센터 고객 발화 텍스트를 보고 고객 성향을 1개로 분류하세요

### 출력 규칙
1. 제시된 성향 키워드 중 가장 적절한 한가지만 선택
2. 설명이나 판단 근거는 절대 출력하지말고 오직 하나의 키워드만 출력한다
3. 여러 개의 성향을 가질 경우 S1 > S2 > S3 > N3 > N2 > N1 의 순서로 우선 순위를 가진다

### 성향 키워드 목록
- N1: 일반형. 큰 특징이 없고 바로 문의사항을 말함
- N2: 수다형. 사적인 이야기나 본인 상황을 길게 설명함
- N3: 신중형. 신중하고 의심을 보임
- S1: 급한성격형. 빠른 처리를 선호함
- S2: 이해부족형. 설명을 잘 이해하지 못하여 반복적으로 확인함
- S3: 불만형. 분노, 짜증을 드러냄
"""

In [8]:
def tokenize_function(examples):
    tokenized_inputs = {
        "input_ids": [],
        "labels": [],
        "attention_mask": []
    }
    
    for script, output in zip(examples['script'], examples['truth']):
        # EXAONE 전용 템플릿
        prompt = f"[|system|]\n{SYSTEM_PROMPT}[|user|]\n{script}[|assistant|]\n"
        answer = f"{output}[|end|]"
        
        # 프롬프트와 답변을 각각 토크나이즈 (패딩 없이)
        p_tokens = tokenizer(prompt, truncation=True, max_length=1024)
        a_tokens = tokenizer(answer, truncation=True, max_length=1024)
        
        # 합치기 (최대 길이 제한)
        input_ids = (p_tokens["input_ids"] + a_tokens["input_ids"])[:1024]
        # Labels 생성: 프롬프트 영역은 -100으로 마스킹 (Loss 계산 제외)
        labels = ([-100] * len(p_tokens["input_ids"]) + a_tokens["input_ids"])[:1024]
        
        # 수동 패딩 처리 (모든 시퀀스를 1024로 맞춤)
        padding_len = 1024 - len(input_ids)
        if padding_len > 0:
            input_ids += [tokenizer.pad_token_id] * padding_len
            labels += [-100] * padding_len # 패딩 영역도 Loss 계산 제외
            
        attention_mask = [1] * (1024 - padding_len) + [0] * padding_len
        
        tokenized_inputs["input_ids"].append(input_ids)
        tokenized_inputs["labels"].append(labels)
        tokenized_inputs["attention_mask"].append(attention_mask)
        
    return tokenized_inputs

In [ ]:
df = pd.read_csv("train.csv")
dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)

In [10]:
dataset

Dataset({
    features: ['id', 'script', 'truth'],
    num_rows: 4800
})

In [11]:
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    optim="paged_adamw_32bit",
    remove_unused_columns=False,
    warmup_ratio=0.1,
)

# Trainer 실행
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,4.070800
20,3.796000
30,3.096800
40,2.446200
50,1.756600
60,1.410000
70,1.320700
80,1.270700
90,1.249000
100,1.184500


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=900, training_loss=1.0717288197411432, metrics={'train_runtime': 7119.6385, 'train_samples_per_second': 2.023, 'train_steps_per_second': 0.126, 'total_flos': 6.55452744450048e+17, 'train_loss': 1.0717288197411432, 'epoch': 3.0})

In [12]:
# 모델 저장
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"학습 완료. 모델이 {output_dir}에 저장되었습니다.")

학습 완료. 모델이 ./exaone-customer-anlysis에 저장되었습니다.


In [ ]:
# 로컬에 저장된 모델과 토크나이저 업로드
repo_id = 'ansui/customer-analysis'

trainer.model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

base_model_id = 'LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct'
adapter_model_id = "ansui/customer-analysis"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)

# 베이스 모델 4-bit 로드
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# LoRA 어댑터 입히기
model = PeftModel.from_pretrained(base_model, adapter_model_id)
model.eval()

# 테스트 함수 정의
def analyze_customer(script):
    prompt = f"[|system|]\n{SYSTEM_PROMPT}[|user|]\n{script}[|assistant|]\n"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=10, 
            # do_sample=False일 때는 temperature를 넣지 않아야 경고가 안 납니다.
            do_sample=False, 
            eos_token_id=tokenizer.eos_token_id
        )
    
    # 생성된 텍스트만 디코딩 (skip_special_tokens=True를 해도 [|end|]가 남을 수 있음)
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # assistant 뒤쪽 답변만 추출
    if "[|assistant|]" in full_text:
        result = full_text.split("[|assistant|]")[-1]
    else:
        result = full_text.replace(prompt, "")
        
    # 특수 토큰 [|end|] 및 공백 깔끔하게 제거
    result = result.replace("[|end|]", "").strip()
    
    # 혹시라도 남을 수 있는 잔여 태그 제거 (예: [|)
    if "[" in result:
        result = result.split("[")[0].strip()
        
    return result


test_script = "네, 카드 결제가 자동 결제인데 오늘 아직 안 됐거든요? 예, 맞습니다. 언제쯤 될까요? 네, 빨리 좀 해주세요. 아 네, 그럼 그렇게 해 주세요. 다음 날이죠?"

print(f"분류 결과: {analyze_customer(test_script)}")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/37.8M [00:00<?, ?B/s]

분류 결과: S1


In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

base_model_id = 'LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct' 
adapter_model_id = "ansui/customer-analysis"
new_repo_id = "ansui/customer-analysis-merged"

# 베이스 모델과 토크나이저 로드
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.bfloat16,
    device_map="cpu",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(adapter_model_id, trust_remote_code=True)

# 어댑터 연결
model = PeftModel.from_pretrained(base_model, adapter_model_id)

# 모델 병합
print("Merging layers...")
merged_model = model.merge_and_unload()

# 허깅페이스에 병합된 모델 업로드
print("Pushing to Hub...")
merged_model.push_to_hub(new_repo_id)
tokenizer.push_to_hub(new_repo_id)

print("업로드 완료")